# Linear Model Tests
This notebook generates a lasso model and produces scores for each of the countries. The scores are the train/test MAEs and the "MPEs" (Mean Percentage Error: 100*MAE/Population).

Now using standard scaler for the models

In [1]:
import pickle
import os
import urllib.request
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Importing the Training Data

In [2]:
# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
# Local files
data_path = 'examples/predictors/ryan_predictor/data'
DATA_FILE = data_path + '/OxCGRT_latest.csv'

if not os.path.exists(data_path):
    os.mkdir(data_path)
urllib.request.urlretrieve(DATA_URL, DATA_FILE)

('examples/predictors/ryan_predictor/data/OxCGRT_latest.csv',
 <http.client.HTTPMessage at 0x7fc1dab03e50>)

In [3]:
df = pd.read_csv(DATA_FILE, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)
# df[cases_df['RegionName'] == 'California']

In [4]:
HYPOTHETICAL_SUBMISSION_DATE = np.datetime64("2020-07-31")
df = df[df.Date <= HYPOTHETICAL_SUBMISSION_DATE]

In [5]:
# Add RegionID column that combines CountryName and RegionName for easier manipulation of data
df['GeoID'] = df['CountryName'] + '__' + df['RegionName'].astype(str)

In [6]:
# Add new cases column
df['NewCases'] = df.groupby('GeoID').ConfirmedCases.diff().fillna(0)

In [7]:
# Keep only columns of interest
id_cols = ['CountryName',
           'RegionName',
           'GeoID',
           'Date']
cases_col = ['NewCases']
npi_cols = ['C1_School closing',
            'C2_Workplace closing',
            'C3_Cancel public events',
            'C4_Restrictions on gatherings',
            'C5_Close public transport',
            'C6_Stay at home requirements',
            'C7_Restrictions on internal movement',
            'C8_International travel controls',
            'H1_Public information campaigns',
            'H2_Testing policy',
            'H3_Contact tracing',
            'H6_Facial Coverings']
df = df[id_cols + cases_col + npi_cols]

In [8]:
# Fill any missing case values by interpolation and setting NaNs to 0
df.update(df.groupby('GeoID').NewCases.apply(
    lambda group: group.interpolate()).fillna(0))

In [9]:
# Fill any missing NPIs by assuming they are the same as previous day
for npi_col in npi_cols:
    df.update(df.groupby('GeoID')[npi_col].ffill().fillna(0))

## Making the Model

In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

scores_df = pd.DataFrame(columns = ['Country', 'TrainMAE', 'TestMAE'])

# Helpful function to compute mae
def mae(pred, true):
    return np.mean(np.abs(pred - true))

do_not_scale_list = ['India', 'Mauritania', 'Philippines', 'Costa Rica']
forest_list = ['Italy', 'Egypt', 'Iraq', 'Singapore', 'Poland', 'Pakistan'
    'Germany', 'Peru', 'Central African Republic', 'Guinea', 'Palestine',
    'France', 'Ecuador', 'Tanzania', 'Kyrgyz Republic']
# The models for these countries were found to perform significantly better using:
# - unscaled data for a linear regression
# and 
# - random forest

scaler = StandardScaler()
model = Lasso(alpha=0.1, precompute=True, max_iter=10000, positive=True, selection='random')
for country in df['CountryName'].unique().tolist():
    
    country_df = df[df['CountryName'] == country]
    
    # Set number of past days to use to make predictions
    nb_lookback_days = 30

    # Create training data across all countries for predicting one day ahead
    X_cols = cases_col + npi_cols
    y_col = cases_col
    X_samples = []
    y_samples = []
    geo_ids = country_df.GeoID.unique()
    for g in geo_ids:
        gdf = country_df[country_df.GeoID == g]
        all_case_data = np.array(gdf[cases_col])
        all_npi_data = np.array(gdf[npi_cols])

        # Create one sample for each day where we have enough data
        # Each sample consists of cases and npis for previous nb_lookback_days
        nb_total_days = len(gdf)
        for d in range(nb_lookback_days, nb_total_days - 1):
            X_cases = all_case_data[d-nb_lookback_days:d]

            # Take negative of npis to support positive
            # weight constraint in Lasso.
            X_npis = -all_npi_data[d - nb_lookback_days:d]

            # Flatten all input data so it fits Lasso input format.
            X_sample = np.concatenate([X_cases.flatten(),
                                       X_npis.flatten()])
            y_sample = all_case_data[d + 1]
            X_samples.append(X_sample)
            y_samples.append(y_sample)

    X_samples = np.array(X_samples)
    y_samples = np.array(y_samples).flatten()
    
    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_samples, y_samples, test_size=0.2, random_state=42)
    
    if country in do_not_scale_list:
        model = Lasso(alpha=0.1, precompute=True, max_iter=10000, positive=True, selection='random')
        model.fit(X_train, y_train)
    elif country in forest_list:
        model = RandomForestRegressor(max_depth=2, random_state=0)
        model.fit(X_train, y_train)
    else:
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        model = Lasso(alpha=0.1, precompute=True, max_iter=10000, positive=True, selection='random')
        model.fit(X_train, y_train)
    
    # Evaluate model
    train_preds = model.predict(X_train)
    train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
#     print('Train MAE:', mae(train_preds, y_train))

    
    test_preds = model.predict(X_test)
    test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
#     print('Test MAE:', mae(test_preds, y_test))
    
    score_df = pd.DataFrame([[country,
                              mae(train_preds, y_train),
                              mae(test_preds, y_test)]],
                            columns=['Country', 'TrainMAE', 'TestMAE'])
    scores_df = scores_df.append(score_df)
    
scores_df

/Users/rlew/opt/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.enet_coordinate_descent_gram(
/Users/rlew/opt/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.enet_coordinate_descent_gram(
/Users/rlew/opt/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.enet_coordinate_descent_gram(
/Users/rlew/opt/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not 

,Country,TrainMAE,TestMAE
0,Aruba,0.672457,0.876852
0,Afghanistan,49.808394,51.384374
0,Angola,3.052795,3.637305
0,Albania,6.878120,7.775907
0,Andorra,2.913342,6.976815
...,...,...,...
0,Vanuatu,0.000000,0.000000
0,Yemen,5.674688,9.139951
0,South Africa,524.722554,503.230442
0,Zambia,27.407808,36.114197


In [12]:
og_df = pd.read_csv(DATA_FILE, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)
og_df['GeoID'] = og_df['CountryName'] + '__' + og_df['RegionName'].astype(str)
geoid_cases = og_df.groupby('GeoID').agg({'ConfirmedCases':np.median}).reset_index()
geoid_cases = geoid_cases.merge(og_df[['GeoID','CountryName']], how='left', left_on='GeoID', right_on='GeoID')
geoid_cases = geoid_cases.groupby('CountryName').agg({'ConfirmedCases':np.sum}).reset_index()
geoid_cases

,CountryName,ConfirmedCases
0,Afghanistan,11497024.0
1,Albania,992288.0
2,Algeria,5456000.0
3,Andorra,300960.0
4,Angola,121792.0
...,...,...
179,Venezuela,2376000.0
180,Vietnam,124960.0
181,Yemen,439296.0
182,Zambia,574464.0


In [13]:
scores_df = scores_df.merge(geoid_cases, how='left', left_on='Country', right_on='CountryName').drop(['CountryName'], axis=1)
scores_df

,Country,TrainMAE,TestMAE,ConfirmedCases
0,Aruba,0.672457,0.876852,36960.0
1,Afghanistan,49.808394,51.384374,11497024.0
2,Angola,3.052795,3.637305,121792.0
3,Albania,6.878120,7.775907,992288.0
4,Andorra,2.913342,6.976815,300960.0
...,...,...,...,...
179,Vanuatu,0.000000,0.000000,0.0
180,Yemen,5.674688,9.139951,439296.0
181,South Africa,524.722554,503.230442,66167904.0
182,Zambia,27.407808,36.114197,574464.0


In [14]:
scores_df['TrainMPE'] = 100*scores_df['TrainMAE']/scores_df['ConfirmedCases']
scores_df['TestMPE'] = 100*scores_df['TestMAE']/scores_df['ConfirmedCases']
scores_df.sort_values(by='TestMPE').reset_index()

,index,Country,TrainMAE,TestMAE,ConfirmedCases,TrainMPE,TestMPE
0,174,United States,203.324825,187.162296,1.993834e+09,0.000010,0.000009
1,23,Brazil,397.553988,428.535856,1.274514e+09,0.000031,0.000034
2,29,Canada,30.143522,31.084708,7.761002e+07,0.000039,0.000040
3,61,United Kingdom,154.317092,160.538823,2.023498e+08,0.000076,0.000079
4,139,Russia,310.372443,315.057313,2.370945e+08,0.000131,0.000133
...,...,...,...,...,...,...,...
179,98,Lesotho,2.967243,3.803760,1.232000e+04,0.024085,0.030875
180,145,Solomon Islands,0.000000,0.000000,0.000000e+00,NaN,NaN
181,163,Turkmenistan,0.000000,0.000000,0.000000e+00,NaN,NaN
182,165,Tonga,0.000000,0.000000,0.000000e+00,NaN,NaN


In [15]:
scores_df.sort_values(by='TestMPE').reset_index().to_csv('case_pred_errors_as_percent.csv', index=False)

In [16]:
scores_df = scores_df.sort_values(by='TestMPE').reset_index()

In [17]:
lin_no_scale = pd.read_csv('lin_vs_rand_forest.csv')
lin_no_scale[['Country','TrainMAE_x','TestMAE_x','ConfirmedCases']]

,Country,TrainMAE_x,TestMAE_x,ConfirmedCases
0,United States,203.377205,187.260836,1.988169e+09
1,Brazil,397.563187,428.478188,1.263662e+09
2,Canada,30.131341,31.077376,7.738953e+07
3,United Kingdom,154.302454,160.560882,2.016495e+08
4,Russia,310.542012,314.734937,2.364210e+08
...,...,...,...,...
175,Bahamas,2.040444,2.270344,3.650400e+04
176,Zambia,27.409937,36.221025,5.728320e+05
177,Kyrgyz Republic,250.130037,182.012063,2.489994e+06
178,Gambia,1.538566,2.641466,2.000700e+04


In [18]:
lin_no_scale[['Country','TrainMAE_x','TestMAE_x','ConfirmedCases']].merge(scores_df, how='left', left_on='Country', right_on='Country')

,Country,TrainMAE_x,TestMAE_x,ConfirmedCases_x,index,TrainMAE,TestMAE,ConfirmedCases_y,TrainMPE,TestMPE
0,United States,203.377205,187.260836,1.988169e+09,174,203.324825,187.162296,1.993834e+09,0.000010,0.000009
1,Brazil,397.563187,428.478188,1.263662e+09,23,397.553988,428.535856,1.274514e+09,0.000031,0.000034
2,Canada,30.131341,31.077376,7.738953e+07,29,30.143522,31.084708,7.761002e+07,0.000039,0.000040
3,United Kingdom,154.302454,160.560882,2.016495e+08,61,154.317092,160.538823,2.023498e+08,0.000076,0.000079
4,Russia,310.542012,314.734937,2.364210e+08,139,310.372443,315.057313,2.370945e+08,0.000131,0.000133
...,...,...,...,...,...,...,...,...,...,...
175,Bahamas,2.040444,2.270344,3.650400e+04,17,2.042852,2.253337,3.660800e+04,0.005580,0.006155
176,Zambia,27.409937,36.221025,5.728320e+05,182,27.407808,36.114197,5.744640e+05,0.004771,0.006287
177,Kyrgyz Republic,250.130037,182.012063,2.489994e+06,89,143.962036,86.048405,2.497088e+06,0.005765,0.003446
178,Gambia,1.538566,2.641466,2.000700e+04,65,1.544397,2.679143,2.006400e+04,0.007697,0.013353


In [19]:
lin_no_scale[['Country','TrainMAE_x','TestMAE_x','ConfirmedCases']].merge(
    scores_df, how='left', left_on='Country', right_on='Country').to_csv('lin_noscale_v_scale.csv')

In [22]:
top_50_country_list = scores_df.sort_values(by='TestMPE')['Country'].tolist()[:50]

In [21]:
import pickle
pickle.dump(top_50_country_list, open( "top_50_country_list.p", "wb" ) )

## Evaluating the Scores

In [13]:
scores_df = scores_df[scores_df['TestMAE'] != 0].sort_values(by='TestMAE')

In [14]:
country_pops = pd.read_csv('countrypops.csv')
country_pops = country_pops[['Country', 'Population']]

In [15]:
scores_df.head()

,Country,TrainMAE,TestMAE
0,Timor-Leste,0.254475,0.152499
0,Greenland,0.129370,0.173346
0,Macao,0.257970,0.189839
0,Dominica,0.177353,0.193705
0,Papua New Guinea,0.598286,0.202319


In [16]:
scores_w_pops = scores_df.merge(country_pops, how = 'left', left_on = 'Country', right_on = 'Country')

In [17]:
population_list = scores_w_pops['Population'].tolist()
for i, val in enumerate(population_list):
    if type(val) not in [int, float]:
        population_list[i] = float(val.replace(',',''))
#     else:
#         population_list[i] = 0
scores_w_pops['Population'] = population_list

In [18]:
#MPE = Mean Percentage Error (I made this term up)
scores_w_pops['TrainMPE'] = 100*scores_w_pops['TrainMAE']/scores_w_pops['Population']
scores_w_pops['TestMPE'] = 100*scores_w_pops['TestMAE']/scores_w_pops['Population']
scores_w_pops[['Country','TestMAE','Population', 'TestMPE']].sort_values(by='TestMPE')

,Country,TestMAE,Population,TestMPE
4,Papua New Guinea,0.202319,8947024.0,0.000002
26,Vietnam,2.425364,97338579.0,0.000002
6,Laos,0.246404,7275560.0,0.000003
29,Myanmar,2.890321,54409800.0,0.000005
16,Taiwan,1.729714,23816775.0,0.000007
...,...,...,...,...
97,Democratic Republic of Congo,23.371566,NaN,NaN
104,Palestine,27.094452,NaN,NaN
106,Cote d'Ivoire,27.507111,NaN,NaN
120,Czech Republic,45.966915,NaN,NaN


In [ ]:
scores_w_pops['TrainMPE'] = 100*scores_w_pops['TrainMAE']/scores_w_pops['Population']
scores_w_pops['TestMPE'] = 100*scores_w_pops['TestMAE']/scores_w_pops['Population']

In [19]:
scores_w_pops.sort_values(by='TestMPE').to_csv('case_pred_errors_as_percent.csv')